In [31]:
from transformers import AutoImageProcessor, AutoModelForSemanticSegmentation
import matplotlib.pyplot as plt
import os
import utils
from dataset import XRayDataset, get_xray_classes
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import torch.nn.functional as F

In [32]:
def encode_mask_to_rle(mask):
    '''
    mask: numpy array binary mask 
    1 - mask 
    0 - background
    Returns encoded run length 
    '''
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# RLE로 인코딩된 결과를 mask map으로 복원합니다.

def decode_rle_to_mask(rle, height, width):
    s = rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(height * width, dtype=np.uint8)
    
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    
    return img.reshape(height, width)

In [33]:
model_dir_path = '/data/ephemeral/home/Dongjin/level2-cv-semanticsegmentation-cv-02-lv3/Baseline/Dongjin/transformers_1120/trained_models/upernet-convnext-small_cont_size_1024'

conf_path = os.path.join(model_dir_path, 'exp.json')
conf = utils.read_json(conf_path)
conf['step_batch_size'] = 1

saved_model_dir_path = utils.saved_model_dir_path(model_dir_path)
image_processor = AutoImageProcessor.from_pretrained(saved_model_dir_path)
model = AutoModelForSemanticSegmentation.from_pretrained(saved_model_dir_path).to(conf['device'])

train_dataset = XRayDataset(mode='train', 
                            transforms=None, 
                            image_processor=image_processor,
                            data_dir_path=conf['data_dir_path'],
                            data_info_path=conf['train_json_path'])

valid_dataset = XRayDataset(mode='valid', 
                            transforms=None, 
                            image_processor=image_processor,
                            data_dir_path=conf['data_dir_path'],
                            data_info_path=conf['valid_json_path'])

test_dataset = XRayDataset(mode='test', 
                            transforms=None, 
                            image_processor=image_processor,
                            data_dir_path=conf['data_dir_path'],
                            data_info_path=conf['test_json_path'])

train_loader = DataLoader(train_dataset, batch_size=conf['step_batch_size'], shuffle=True, num_workers=conf['num_workers'])
valid_loader = DataLoader(valid_dataset, batch_size=conf['step_batch_size'], shuffle=False, num_workers=conf['num_workers'])
test_loader = DataLoader(test_dataset, batch_size=conf['step_batch_size'], shuffle=False, num_workers=conf['num_workers'])

In [ ]:

def test(model, data_loader, thr=0.5):
    xray_classes = get_xray_classes()
    model.eval()

    rles = []
    filename_and_class = []
    with torch.no_grad():
        for batch, image_names in tqdm(data_loader, total=len(data_loader)):
            inputs = batch["pixel_values"].to('cuda')
            outputs = model(inputs).logits
            
            outputs = F.interpolate(outputs, size=(2048, 2048), mode="bilinear")
            outputs = torch.sigmoid(outputs)
            outputs = (outputs > thr).detach().cpu().numpy()
            
            for output, image_name in zip(outputs, image_names):
                for c, segm in enumerate(output):
                    rle = encode_mask_to_rle(segm)
                    rles.append(rle)
                    filename_and_class.append(f"{xray_classes['idx2class'][c]}_{image_name}")
                    
    return rles, filename_and_class

In [35]:
rles, filename_and_class = test(model, test_loader, thr=0.5)

  0%|          | 0/288 [00:00<?, ?it/s]


AttributeError: 